In [ ]:
!pip install plotly dash
!pip install jupyter_dash
!pip install dash_bootstrap_components

In [1]:

import numpy as np
import pandas as pd  # To read data
import dash 
import dash_bootstrap_components as dbc
from jupyter_dash import JupyterDash
from dash import dcc, html, Input, Output, dash_table
import plotly.express as px


In [2]:


# -----------------------------------------------------------
# 1) Load and Processing Data NBA Players: by Justinas Cirtautas
# -----------------------------------------------------------
nba_player_data = pd.read_csv("Data/all_seasons.csv")

# Unique NBA Seasons for the slider
nba_player_data = nba_player_data.sort_values(by='season')

american_nba_players=nba_player_data[nba_player_data["country"]=="USA"]

international_nba_players=nba_player_data[nba_player_data["country"]!="USA"]

# Count players by country and season
players_by_country_and_season = international_nba_players.groupby(["season", "alpha-3","country"])["player_id"].count().reset_index()

# Renaming columns
players_by_country_and_season.columns = ["Season","Iso Alpha", "Country", "Total Players"]

# Count players by continet and season
players_by_continent_and_season = international_nba_players.groupby(["season","region"])["player_id"].count().reset_index()

#Renaming Columns
players_by_continent_and_season.columns=["Season", "Continent", "Total Players"]
#Getting unique seasons
seasons = players_by_country_and_season["Season"].unique()


# getting all american players
american_nba_players["Region"]="USA"
# getting all international players
international_nba_players["Region"]="International"
# getting every second season for line chart x axis labels
tick_vals = nba_player_data["season"][::2]


# -----------------------------------------------------------
# 2) Create Dash App with Bootstrap Styling
# -----------------------------------------------------------
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.LUX])
server = app.server  # Expose Flask app for deployment

# -----------------------------------------------------------
# 3) Layout and Components
# -----------------------------------------------------------
app.layout = dbc.Container([
    html.H2("NBA Player Statistics (1996-2022)", className="mt-4 mb-4 text-center"),
    html.H4("Data is From the \"Nba Players\" dataset by Justinas Cirtautas", className="mt-4 mb-4 text-center"),
    html.H4("(Dataset is available on Kaggle )", className="mt-4 mb-4 text-center"),
    
    dcc.Tabs(id="tabs-example", children=[

        # ------ Tab 1: Nba International Players  ------
        dcc.Tab(label="Emergence Of NBA International Players", children=[
            html.H2("Emergence Of NBA International Players", className="mt-4 mb-4 text-center"),
            html.Br(),
            html.P("The NBA is a basketball league dominated by US players that have played college basketball "
                 "but there is more of an emergence of international players entering the league in recent years.",
                   style={
                        "color": "black"       # sets text color to black
                       
                        }
                  ),
            html.P("I researched into the rise of international players entering the league from 1996-2022"
                  " and also which county and continents are producing the most NBA players throughout the years.  ",
                   style={
                        "color": "black"        # sets text color to black
                       
                        }),
           html.P("Note: some players came from countries that dont exist today and were mapped to a different country."
                 "For example, USSR to Russia, Bosnia to Bosnia and Herzegovnia, Yugoslavia to Serbia, "
                 "Serbia and Montenegro to Serbia.",
                 style={
                        "color": "black"       # sets text color to black
                        
                        }),
                    
            html.Br(),
            html.P("Discover Number Of Players From Each Country By Pressing Play Or Drag The Slider To Your Selected Season.",
                  style={
                        "color": "black"       # sets text color to black
                        
                        }),
            html.P("Select A Season Below (1996 = 96/97 season, 1997 = 97/98 season, etc.)",
                  style={
                        "color": "black"        # sets text color to black
                        
                        }),
            #chloropleth graph
            dcc.Graph(id="choropleth-map", style={"height": "550px"}),
            html.Br(),
            html.P("In the map above, there was a very small amount of international NBA players in the 1996 season." 
                  "Fast forward to the 2022 season and there is more NBA players coming from a wide variety"
                   " of countries around the world.",
                   style={
                        "color": "black"        # sets text color to black
                       
                        }
                  ),
           
            html.Br(),
            html.P("Select A Season Below (1996 = 96/97 season, 1997 = 97/98 season, etc.)",
                  style={
                        "color": "black"        # sets text color to black
                        
                        }),
            html.Br(),
            #creating season slider
            dcc.Slider(
                id="year-slider",\
                min=min(seasons),
                max=max(seasons),
                value=min(seasons),
                marks={str(season): str(season) for season in seasons},
            ),
            #row that contains continenents and country bar chart.
            dbc.Row([
                dbc.Col(dcc.Graph(id="players-per-continent-bar-chart"), width=6),
                dbc.Col(dcc.Graph(id="top-international-countries-bar-chart"), width=6),
            ], className="mb-4"),
            html.Br(),
            html.P("Throughout the years, most of the international NBA Players are originating from Europe, "
                  "we can also see in the countries barchart that there is more countries and the number of players are increasing"
                  " in certain countries as the years go by.",
                  style={
                        "color": "black"        # sets text color to black
                        
                        }),
            html.Br(),
            dcc.Graph(id="international-players-line-chart", style={"height":"400px"}),
            html.Br(),
            dcc.Graph(id="continent-players-line-chart", style={"height":"400px"}),
            html.P("As we can see that there is a steady increase of the number of International"
                  " players entering the league between 1996 - 2022.",
                  style={
                        "color": "black"        # sets text color to black
                       
                        }),
            html.P("We can see a sharp increase of NBA International Players after 2001. "
                  "This could be due to the Basketball Without Borders program that was founded "
                  "in 2001 by the NBA and FIBA that provide training camps to increase the potential "
                  "of international basketball players and also promote the sport in other regions",
                  style={
                        "color": "black"        # sets text color to black
                        
                        }),
            html.P("In the second line graph, Europe has the biggest increase of players playing in the NBA. "
                  "This is probably from Euroleague players that are being spotted by NBA scouts. "
                  "The EuroLeague is considered to be the second biggest professional basketball league"
                  "behind the NBA.",
                  style={
                        "color": "black"       # sets text color to black
                       
                        }),
            html.Br(),
            html.P("From 1996 to 2022, there has been a big increase of international players joining the "
                  "NBA from a wider variety of countries with the help of the Basketball Witout Borders program "
                  "making it easier to spot prospects and with the increasing popularity of the NBA around the world. ",
                  style={
                        "color": "black"        # sets text color to black
                        
                        }),
        ]),
        #Age, Height and Weight Of Nba Players Over Time
        dcc.Tab(label="Age, Height and Weight Of Nba Players Over Time", children=[
           html.H2("Age, Height and Weight Of Nba Players Over Time", className="mt-4 mb-4 text-center"),
            html.P("This tab aims to discover trends in NBA players average weight, height and age throughout the years.",
                  style={
                        "color": "black"        # sets text color to black
                        
                        }),
            html.P("Feel free to interact with the graphs below to see the trends in player's weight, height,"
                  " and age yourself.",
                  style={
                        "color": "black"        # sets text color to black
                        
                        }),
            html.Br(),
            html.P("Filter by Age, Weight and Height",
                  style={
                        "color": "black"       # sets text color to black
                        
                        }),
            #Creating a dropdown menu
            dcc.Dropdown(
                id="age-height-weight-dropdown",
                options=["Age","Weight (Kg)", "Height (Cm)"],
                value="Age",
                clearable=False,
            ),
            html.Br(),
            dcc.Graph(id="players-age-weight-height-line-chart"),
            html.Br(),
            html.P("Throughout the years, we can see that the average weight, height and age "
                  "of players are dropping. Evolving training methods, increased skill level and teams favouring younger "
                  "players that are able to keep up with the physical demands of playing in the NBA"
                   " could be key factors to this declining trend.",
                  style={
                        "color": "black"        # sets text color to black
                        
                        }),
            html.Br(),
            html.P("Select A Season Below (1996 = 96/97 season, 1997 = 97/98 season, etc.)",
                  style={
                        "color": "black"        # sets text color to black
                        
                        }),
            html.Br(),
            #creating Season slider
            dcc.Slider(
                id="year-slider-histogram",
                min=min(seasons),
                max=max(seasons),
                value=min(seasons),
                marks={str(season): str(season) for season in seasons},
            ),
            html.Br(),
            # Creating a histogram graph
            dcc.Graph(id="age-histogram", style={"height": "400px"}),
            html.Br(),
            html.P("In 1996, there is a higer distribution of players aged between 25 - 29. "
                  "Fast forward to 2022 and there is more total players in the league. "
                  "There is also higer distribution of players aged 20 - 24 years old. "
                  "There is also more players aged between 25 - 29 years old compared to "
                  "the 1996 season. It seems that teams are favoring younger players and "
                  "and the longetivity of players has decreased as there is more players aged "
                  "between 30 - 44 years old in 1996 compared to the 2022 season.",
                  style={
                        "color": "black"        # sets text color to black
                        
                        }),
            html.Br(),
            html.P("Throughout the years in the NBA, the average weight, height and age of players "
                  "are decreasing. Factors that could be playing a part in this trend is the evolving "
                  "training methods and more emphasis on skill level. The increasing physical demands of the NBA "
                  "could be a factor as to why there is more younger players in 2022 and a decrease in longetivity "
                  "of players compared to the 1996 season.",
                  style={
                        "color": "black"        # sets text color to black
                        
                        }),
            html.Br(),
           
            
        ]),
        # International Players Vs USA Players (Points, Assists and Rebounds.) Tab
         dcc.Tab(label="International Players Vs USA Players (Points, Assists and Rebounds.)", children=[
             html.H2("International Players Vs USA Players (Points, Assists and Rebounds.)", className="mt-4 mb-4 text-center"),
            html.Br(),
             html.P("This tab aims to discover trends amongst the top 10 USA players and the Top 10 international "
                   "players in points, assists, rebounds from each season. I got the average in each category "
                   "from the top 10 USA and international players. Feel free to see the trends below. ",
                   style={
                        "color": "black"        # sets text color to black
                       
                        }),
             html.Br(),
            html.P("Filter by Points, Assists, Rebounds",
                  style={
                        "color": "black"       # sets text color to black
                            
                        }),
            dcc.Dropdown(
                id="points-assists-rebounds-dropdown",
                options=["Points","Assists", "Rebounds"],
                value="Points",
                clearable=False,
            ),
             html.P("Note: there is only 9 international players recorded for the 1996 season.",
                   style={
                        "color": "black"       # sets text color to black
                        
                        }),
            dcc.Graph(id="players-pts-asts-reb-line-chart"),
            html.P("Points: the average amount of points scored from both USA and international players increase over time."
                  "US players averaged more points in the earlier seasons than international players, but the international "
                  " players nearly catch up to the US players in the later seasons.",
                  style={
                        "color": "black"        # sets text color to black
                        
                        }),
             
             html.P("Rebounds: The US players line does't change too much until it dramitically "
                   "decreases after the 2016 season. The international players line increases throughout "
                   "the seasons and eventually passes out the US players at the 2018 season",
                   style={
                        "color": "black"        # sets text color to black
                       
                        }),
             html.P("Assists: The US line slightly decreases over time while the International line"
                   " increases over time but the US players achieved higher assist averages in each "
                   " season.",
                   style={
                        "color": "black"        # sets text color to black
                       
                        }),

             html.Br(),
             html.P("In the line graph above, we can see an increase in points, rebounds and assists for international players throughout "
                   "the seasons, especially in points and rebounds. There is a dramatic increase in points and rebounds "
                   "between the 2016 and 2022 seasons which would be the same time that international superstars "
                   "Luka Dončić, Nikola Jokić (who won the MVP award in the 20/21 and 21/22 seasons), Joel Embiid (who won the MVP award in the 22/23 "
                    "season) and Giannis Antetokounmpo (who won the MVP award in the 18/19 "
                   " and 19/20 seasons) were playing in the league. ",
                   style={
                        "color": "black"        # sets text color to black
                        
                        }),
             html.P("While there is a large increase in NBA international players, "
                    "There is also an increase in skill quality of international "
                    "players entering the league. ",
                   style={
                        "color": "black"        # sets text color to black
                        
                        }),
             
            
            
        ]),
        
    ]),],fluid=True)
    



       

# -----------------------------------------------------------
# 4) Callbacks for Interactivity
# -----------------------------------------------------------

# Callback: Update Chloropleth, barcharts based on season selection, line charts.
@app.callback(
    Output("choropleth-map", "figure"),
    Output("top-international-countries-bar-chart", "figure"),
    Output("players-per-continent-bar-chart", "figure"),
    Output("international-players-line-chart", "figure"),
    Output("continent-players-line-chart","figure"),
    Input("year-slider", "value")
)
def update_choropleth(selected_season):
    # getting international players from players_by_country_and_season dataframe
    df_internationl_players = players_by_country_and_season[players_by_country_and_season["Country"] != "USA"]
    #getting players from continet based on season
    df_continent_filtered= players_by_continent_and_season[players_by_continent_and_season["Season"] == selected_season]
    #getting max value of international players.
    max_value= df_internationl_players["Total Players"].max()
    #creating coropleth chart
    fig = px.choropleth(
        df_internationl_players,
        locations="Iso Alpha",
        color="Total Players",
        hover_name="Country",
        animation_frame="Season",
        title=f"Number of NBA Players From Around The World",
        # Set the range from 0 to 90 so the color scale remains constant
        range_color=[0, max_value]
    )

    
    # getting international players based on selected season.
    df_internationl_players=df_internationl_players[df_internationl_players["Season"]== selected_season]
    #getting 10 countries with the most players.
    top_10_countries = df_internationl_players.nlargest(10, "Total Players")

    #creating most players in country bar chart.
    fig_most_players_in_country = px.bar(
        top_10_countries,
        x="Iso Alpha", y="Total Players",
        hover_data=["Country"],
        title=f"Most NBA Players Outside Of The USA In {selected_season}.",
        labels={'Iso Alpha': 'Countries'
           }
    )
    fig_most_players_in_country.update_layout(
        template="plotly_white",
        
    )
    fig_most_players_in_country.update_xaxes(tickangle=45)

    #creating most players from each continent bar chart.
    fig_players_in_continent = px.bar(
        df_continent_filtered,
        x="Continent", y="Total Players",
        title=f"Number Of NBA Players From Each Continet In {selected_season}."
    )
    
    fig_players_in_continent.update_layout(
        template="plotly_white",
    )
    fig_players_in_continent.update_xaxes(tickangle=45)
    #getting all international players for line chart.
    international_player_line_chart_data=nba_player_data[nba_player_data["country"]!= "USA"]
    #getting total amount of international players for line chart.
    international_player_line_chart_data=international_player_line_chart_data.groupby(["season"])["player_id"].count().reset_index()
    #renaming columns
    international_player_line_chart_data.columns=["Season","Total Players"]
    # getting every second season for line chart x labels
    tick_vals = international_player_line_chart_data["Season"][::2]
   #creating international line chart.
    fig_international_player_line_chart = px.line(
       international_player_line_chart_data,
        markers=True,
       x="Season", y="Total Players",
       title=f"Number Of International NBA Players Over Time"
   )
    fig_international_player_line_chart.update_layout(
        template="plotly_white",
    xaxis=dict(
        tickmode="array",
        tickvals=tick_vals,  # Show every 5th season
        ticktext=[str(year) for year in tick_vals]  # Convert to string for labels
    ))
    #creating continetal player line chart.
    fig_continental_player_line_chart = px.line(
       players_by_continent_and_season,
        markers=True,
       x="Season", y="Total Players",
       color="Continent",
       title=f"Number Of International NBA Players From Each Continent Over Time"
   )
    fig_continental_player_line_chart.update_layout(
        template="plotly_white",
    xaxis=dict(
        tickmode="array",
        tickvals=tick_vals,  # Show every 2nd season
        ticktext=[str(year) for year in tick_vals]  # Convert to string for labels
    )
    )
        
    return fig, fig_most_players_in_country, fig_players_in_continent,fig_international_player_line_chart,fig_continental_player_line_chart
    
@app.callback(
 
    Output("players-age-weight-height-line-chart","figure"),
    Output("age-histogram","figure"),
    Input("tabs-example", "value"),
    Input("age-height-weight-dropdown", "value"),
    Input("year-slider-histogram","value")
)
def update_age_weight_height_chart(tab, selected_attribute, selected_year):
    
        
    # getting average age of players per season.
    avg_age_per_season = nba_player_data.groupby('season')['age'].mean().reset_index()

    # Renaming columns.
    avg_age_per_season.columns = ['Season', 'Value']

    # getting average height of players per season.
    avg_height_per_season = nba_player_data.groupby('season')['player_height'].mean().reset_index()

    # Renaming columns
    avg_height_per_season.columns = ['Season', 'Value']

    # getting the average weight of players per season
    avg_weight_per_season = nba_player_data.groupby('season')['player_weight'].mean().reset_index()

    # Renaming columns
    avg_weight_per_season.columns = ['Season', 'Value']
    # setting df_filtered dataset based on the selected attribute.
    if selected_attribute=="Age":
        df_filtered=avg_age_per_season
    elif selected_attribute=="Height (Cm)":
        df_filtered=avg_height_per_season
    elif selected_attribute=="Weight (Kg)":
        df_filtered=avg_weight_per_season
      #creating age, height and weight line chart.
    fig_age_height_weight_line_chart=px.line(
        df_filtered,
        markers=True,
       x="Season", y="Value",
       title=f"Distribution of Players {selected_attribute} In The NBA Over Time "
   )
    fig_age_height_weight_line_chart.update_layout(
    xaxis_title="Season",
    yaxis_title=f"{selected_attribute}"
)
    tick_vals = df_filtered["Season"][::2]
    fig_age_height_weight_line_chart.update_layout(
        template="plotly_white",
    xaxis=dict(
        tickmode="array",
        tickvals=tick_vals,  # Show every 5th season
        ticktext=[str(year) for year in tick_vals]  # Convert to string for labels
    )
    )
    df_selected=nba_player_data[nba_player_data["season"]==selected_year]
    total_players=df_selected.shape[0]
    #creating histogram for age distribution based on selected year.
    fig_age_histogram=fig = px.histogram(
    df_selected, 
    x='age', 
    nbins=10, 
    title=f'Age Distribution for NBA Players in {selected_year}(Total Players = {total_players})',
    text_auto=True,
    labels={'age': 'Player\'s Age',
           'count': 'Number of Players'}

)
    fig_age_histogram.update_layout(
        template="plotly_white",
    )
    return fig_age_height_weight_line_chart, fig_age_histogram

    #return fig_most_players_in_country
@app.callback(
    Output("players-pts-asts-reb-line-chart", "figure"),
    Input("points-assists-rebounds-dropdown", "value"),
)
def update_international_USA_stats_line_chart(selected_stat):
    
    #getting the top 10 highest US points scorers and getting the average for each season
    top10_USA_pts_avg= (
    american_nba_players.groupby(['season','Region'])
      .apply(lambda group: group.nlargest(10, 'pts')['pts'].mean())
      .reset_index(name='Value')
    )

    #getting the top 10 highest international points scorers and getting the average for each season
    top10_international_pts_avg= (
    international_nba_players.groupby(['season','Region'])
      .apply(lambda group: group.nlargest(10, 'pts')['pts'].mean())
      .reset_index(name='Value')
    )
    #getting the top 10 highest US rebounders and getting the average for each season
    top10_USA_reb_avg= (
    american_nba_players.groupby(['season','Region'])
      .apply(lambda group: group.nlargest(10, 'reb')['reb'].mean())
      .reset_index(name='Value')
    )

    #getting the top 10 highest international rebounders and getting the average for each season
    top10_international_reb_avg= (
    international_nba_players.groupby(['season','Region'])
      .apply(lambda group: group.nlargest(10, 'reb')['reb'].mean())
      .reset_index(name='Value')
    )
    #getting the top 10 highest US players with assists and getting the average for each season
    top10_USA_ast_avg= (
    american_nba_players.groupby(['season','Region'])
      .apply(lambda group: group.nlargest(10, 'ast')['ast'].mean())
      .reset_index(name='Value')
    )
     #getting the top 10 highest international players with assists and getting the average for each season
    top10_international_ast_avg= (
    international_nba_players.groupby(['season','Region'])
      .apply(lambda group: group.nlargest(10, 'ast')['ast'].mean())
      .reset_index(name='Value')
    )



    
    # setting the df_filtered data frame based on the selected attribute.
    if selected_stat=="Points":
        df_filtered=pd.concat([top10_USA_pts_avg,top10_international_pts_avg ])
    elif selected_stat=="Assists":
        df_filtered=pd.concat([top10_USA_ast_avg,top10_international_ast_avg ])
    elif selected_stat=="Rebounds":
        df_filtered=pd.concat([top10_USA_reb_avg,top10_international_reb_avg ])
    #creating the stats linechart.
    fig_stats_line_chart=px.line(
        df_filtered,
        markers=True,
       x="season", y="Value",
       color="Region",
       title=f"Comparing {selected_stat} Between USA and International Players Over Time "
   )
    
    tick_vals=df_filtered["season"].unique()[::2]
    fig_stats_line_chart.update_layout(
        template="plotly_white",
        xaxis_title="Season",
        yaxis_title=f"{selected_stat}",
        xaxis=dict(
        tickmode="array",
        tickvals=tick_vals,  # Show every 5th season
        ticktext=[str(year) for year in tick_vals]  # Convert to string for labels
    ))
    return fig_stats_line_chart
    
# Run server
if __name__ == "__main__":
    app.run_server(debug=True)


C:\Users\Team Knowhow\AppData\Local\Temp\ipykernel_29300\2176515827.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  american_nba_players["Region"]="USA"
C:\Users\Team Knowhow\AppData\Local\Temp\ipykernel_29300\2176515827.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  international_nba_players["Region"]="International"
